# Aprendizaje Profundo

Hasta ahora todos los modelo que venimos viendo fueron más bien *llanos*, es decir, con pocas capas de procesamiento. Lo más complejo fue un MLP con una sola capa oculta. Si bien existe un teorema que garantiza que cualquier función puede ser aproximada con un grado de error tan bajo como se quiera con una sola capa oculta, este resultado en más bien teórico y sirve para demostrar que con este tipo de modelo no va a existir una limitación intrínseca como en el caso del perceptrón simple. No es un resultado que en la práctica nos pueda decir cuantas unidades en la capa oculta van a ser necesarias, o cuanto tiempo de entrenamiento se requerirá.

Si bien siempre se supo que utilizar más etapas de procesamiento (i.e. más capas ocultas) permitiría encontrar mejores soluciones para problemas más complejos durante bastante tiempo las arquitecturas usadas en aplicaciones reales nunca tenían más de 3 capas ocultas. Y esta limitación no solo era debida a limitaciones en el poder de cómputo de la época sino a una dificultad específica del método de entrenamiento.

Al entrenar por *backprop* las estimaciones del error que se van retropropagando se degradan con cada capa. Es decir cuanto más capas ocultas tenga un modelo (más profunda sea su arquitectura) las estimaciones del error para hacer las correcciones a los pesos en las primeras capas serán cada vez peores.  La revolución en este campo que ocurrió más o menos recientemente fue gracias al desarrollo de algunas técnicas que empezaron a permitir entrenar exitosamente modelos con arquitecturas más profundas.

Empecemos por ver como sería la implementación de un MLP con tantas capas ocultas como querramos.
Para esto vamos a definir una nueva clase cuyo constructor tome como parámetro una lista de enteros con la cantidad de unidades por capa que se deseen utilizar.
En la implementación anterior los parámetros entrenables de la capa oculta y de salida habían sido objetos *Linear* incluídos directamente como miembros de la clase, lo cual garantizaba que la función de optimización sabría que estos debían ser modificados para tratar de reducir la función de costo.  En este caso, como vamos a tener una lista de capas y no podemos usar sencillamente el tipo de *list* nativo de python porque no es derivado de *Module*. En su lugar vamos a tener que utilizar el tipo *ModuleList* provisto por torch que funciona de la misma forma que una lista común pero que permitirá que los parámetros entrenables de sus miembros sean incluídos en el aprendizaje de nuestro modelo.

Notar también que para producir la respuesta de la red estamos utilizando un ciclo *for* nativo de python. Esto no es problema mientras que las operaciones involucradas solo utilicen funciones de torch.  Es decir, no importa como se calcule la salida, torch sabe como construir el grafo de computo.

In [2]:
import torch

class MLP( torch.nn.Module):
    def __init__( _, sizes):
        super().__init__()
        _.layers = torch.nn.ModuleList()
        for i in range(len(sizes)-1):
            _.layers.append( torch.nn.Linear( sizes[i], sizes[i+1]))

    def forward( _, x):
        h = x
        for hidden in _.layers[:-1]:
            h = torch.sigmoid( hidden( h))
        output = _.layers[-1]
        y = output( h)           # Quiero unidades lineales en la salida.
        return y


# Autoencoders

Una de las primeras aplicaciones prácticas de las redes neuronales fue encontrar formas efectivas de reducir la cantidad de dimensiones de datos. Esto implicaba la ventaja de utilizar funciones no-lineales y perder la menor cantidad de información posible.  Los modelos que se destacaron en esto son los llamados *autoencoders* que son sencillamente un MLP entrenados para tratar de reproducir en su salida el mismo patrón que reciben en la entrada, pasando por una capa oculta de (generalmente) menor dimensión.

Con todo lo que vimos hasta ahora nos resulta trivial implementar un modelo de este tipo.

In [3]:
class AE( torch.nn.Module):
    def __init__( _, vsize, hsize):
        super().__init__()
        _.enc = torch.nn.Sequential( torch.nn.Linear( vsize, hsize), torch.nn.Tanh())
        _.dec = torch.nn.Sequential( torch.nn.Linear( hsize, vsize), torch.nn.Tanh())

    def forward( _, x):
        return _.dec( _.enc( x))        # enc y dec se usan como funciones.


Notar que al utilizar *Sequential* para definir la capa oculta y de salida estoy internamente creando dos sub-redes que suelen ser llamadas el *encoder* y *decoder*.  El implementarlo de esta forma me da la ventaja de, después de entrenarlo, poder usarlas independientemente para codificar o decodificar patrones según mis necesidades.

Y finalmente puedo crear lo que se suele llamar un *autoencoder profundo* o *autoencoders apilados* de forma similar a como implementamos arriba el MLP con varias capas ocultas.
En este caso la dimensión de la capa oculta de un autoencoder coincide en dimensión con las capas de entrada y salida del siguiente autoencoder. Esto hace que pueda pasarle como entrada al autoencoder superior el patrón codificado por el autoencoder anterior.
De esta forma es posible ir entrenando los autoencoder progresivamente. Es decir, entreno el primer autoencoder simple solo, una vez que fue entrenado lo uso para encodear los patrones y estos serviran de entrada para el siguiente autoencoder, y así procedo con todos.
Para esto en la implementación incluiremos un parámetro adicional llamado *depth* para indicar hasta que profundidad deseamos entrenar el modelo, o utilizarlo para codificar o decodificar.

In [4]:
class DeepAE( torch.nn.Module):            # Deep Auto-Encoders.
    def __init__( _, sizes):
        super().__init__()
        _.subnet = torch.nn.ModuleList()
        for i in range(len(sizes)-1):
            _.subnet.append( AE(sizes[i],sizes[i+1]))

    def enc( _, x, depth=None):
        depth = len(_.subnet) if depth is None else depth+1
        xi = x
        for i in range(depth):
            xi = _.subnet[i].enc( xi)
        return xi

    def dec( _, y, depth=None):
        depth = len(_.subnet) if depth is None else depth+1
        yi = y
        for i in reversed(range(depth)):
            yi = _.subnet[i].dec( yi)
        return yi

    def forward( _, x, depth=None):     # Varia la profundidad del AE.
        yi = _.enc(  x, depth)
        xi = _.dec( yi, depth)
        return xi, yi


Una técnica similar a esta fue la primera en ser utilizada para poder entrenar arquitecturas realmente profundas.
La idea es que puedo entrenar cada autoencoder independientemente por *backprop* con resultados confiables, y una vez que tengo estos pesos entrenados puedo utilizarlos como punto de partida para entrenar un MLP en cualquier otra tarea.
Si bien no creo que necesitemos utilizar una arquitectura como esta es bueno que se entienda cual era el problema que impedía entrenar eficazmente arquitecturas profundas y cómo esto ayuda a superar esa dificultad.